In [1]:
import requests
from bs4 import BeautifulSoup as bs

## 1. Set-up urls to be scraped

In [2]:
# Set-up dictionary of all Mars urls to be scraped
url_dict = {"NASA":"https://mars.nasa.gov/news/",\
           "JPL":"https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars",\
           "Twitter":"https://twitter.com/marswxreport?lang=en",\
           "space-facts":"https://space-facts.com/mars/",\
           "geology":"https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"}

## 2. Scrape NASA and extract latest news _title_ and _text_

In [53]:
response = requests.get(url = url_dict["NASA"])

soup = bs(response.text, "html.parser")

# print(soup.prettify())

results = soup.find_all("div", class_ = "slide")

mars_latest_news = []

for result in results:    
    try:    
        news_title = result.find("div", class_ = "content_title").text
        news_description = result.find("div", class_ = "rollover_description_inner").text

        news_dict = {'title':news_title.strip(),\
                    'description':news_description.strip()}

        mars_latest_news.append(news_dict)
    except AttributeError as e:
        print(e)        